# Bonus Task

This notebook contains the solution for the task #3, management of a Virtual Power Plant with stochasticity. <br> <br>
For reference:
<br>
Morales, J.M., Conejo, A.J., Madsen, H., Pinson, P. and Zugno, M., 2013. Integrating renewables in electricity
markets: operational problems (Vol. 205). Springer Science & Business Media., Chapter 8.

In [17]:
using JuMP, HiGHS, Plots, DataFrames

In [18]:
model = Model(HiGHS.Optimizer)
#set_silent(model)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

## Compose Model

### Variables and Functions

In [19]:
T = ["t1", "t2", "t3"] # Time steps
Ω = ["ω1", "ω2", "ω3"] # Scenarios

I = [ "i1"] # power plants

Q = ["q1"] # wind generation units

J = ["j1"] # flexible loads (energy consumers)

K = ["k1"] # storage units (batteries etc.)

λ = Dict("t1" => 20, "t2" => 80, "t3" => 45) # Market price --> check
π = Dict("ω1" => 0.2, "ω2" => 0.3, "ω3" => 0.5) # probabilities of scenarios


#aG = 5 #’Quadratic coefficient of GT cost function’ /5/
bG = 10 # ’Linear coefficient of GT cost function’ /10/
cG = 50 #’No-load cost’ /50/

#aL = -30 #’Quadratic coefficient of GF utility function’ /-30/
bL = 50 #’Linear coeffcient of GF utility function’ /150/
cL = 5 #’Constant term of gear factory utility function’ /5/


# Define your PW_values dictionary
PW_values = Dict(
    ("q1", "ω1", "t1") => 2.5, ("q1", "ω1", "t2") => 4.0, ("q1", "ω1", "t3") => 6.0,
    ("q1", "ω2", "t1") => 6.0, ("q1", "ω2", "t2") => 4.0, ("q1", "ω2", "t3") => 3.5,
    ("q1", "ω3", "t1") => 2.0, ("q1", "ω3", "t2") => 1.1, ("q1", "ω3", "t3") => 1.5
)

# Initialize an empty dictionary for PW
PW = Dict()

# Populate the PW dictionary with values from PW_values
for q in Q
    for ω in Ω
        for t in T
            PW[(q, ω, t)] = PW_values[(q, ω, t)]
        end
    end
end

In [20]:
# For time steps
function previous_time_period(t, T)
    index = findfirst(isequal(t), T)
    return T[index - 1]
end


previous_time_period (generic function with 1 method)

In [21]:
@variables(model, begin
        
    PD[T] # power exchanged with the electricity market at time step t
    EG_i[I,Ω,T] >= 0 # energy generated by power plant i in scenario ω and time step t
    #EG_j[J,Ω,T] >= 0 # energy consumed by flexbile load j in scenario ω and time step t
    #C[I] # flexbile costs per unit of electricity generated by power plant i 
    C[I,Ω,T] >= 0  # Cost variable for each i, ω, t
    #U[J] # price per unit of energy bought by flexbile load j
    U[J,Ω,T] >= 0
  #  PW[Q,Ω,T] >= 0 # energy generated by wind generation unit q in scenario ω and time step t ------------------------> depends on the scenario!!!
    PW_curt[Q,Ω,T] >= 0 # curtailment of energy generated by wind generation unit q in scenario ω and time step t
    P_discharge_S[K,Ω,T] >= 0 # power discharged from storage unit k in scenario ω and time step t
    P_charge_S[K,Ω,T] >= 0 # power charged to storage unit k in scenario ω and time step t
    ES[K,Ω,T] >= 0 # energy status of storage unit k in scenario ω and time step t
    EL[J,Ω,T] >= 0 # electricity consumed by flexible load j in scenario ω and time step t
    v[I,Ω,T], Bin # binary variable that declares whether or not power plant i is on/off at time step t
    
end)

(1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
And data, a 3-element Vector{VariableRef}:
 PD[t1]
 PD[t2]
 PD[t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{VariableRef, 3}:
[:, :, "t1"] =
 EG_i[i1,ω1,t1]  EG_i[i1,ω2,t1]  EG_i[i1,ω3,t1]

[:, :, "t2"] =
 EG_i[i1,ω1,t2]  EG_i[i1,ω2,t2]  EG_i[i1,ω3,t2]

[:, :, "t3"] =
 EG_i[i1,ω1,t3]  EG_i[i1,ω2,t3]  EG_i[i1,ω3,t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{VariableRef, 3}:
[:, :, "t1"] =
 C[i1,ω1,t1]  C[i1,ω2,t1]  C[i1,ω3,t1]

[:, :, "t2"] =
 C[i1,ω1,t2]  C[i1,ω2,t2]  C[i1,ω3,t2]

[:, :, "t3"] =
 C[i1,ω1,t3]  C[i1,ω2,t3]  C[i1,ω3,t3], 3-dimensional DenseAxisArray{VariableRef,3,...} with index sets:
    

### Constraints

In [22]:
# Energy balance constraint
@constraint(model, E1[ω in Ω, t in T], 
    sum( EG_i[i,ω,t] for i in I) + (sum( PW[q,ω,t] - PW_curt[q,ω,t] for q in Q) + sum( P_discharge_S[k,ω,t] for k in K))
    == sum( EL[j,ω,t] for j in J) + (sum( P_charge_S[k,ω,t] for k in K) + PD[t])
)


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, ["ω1", "ω2", "ω3"]
    Dimension 2, ["t1", "t2", "t3"]
And data, a 3×3 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 E1[ω1,t1] : -PD[t1] + EG_i[i1,ω1,t1] - PW_curt[q1,ω1,t1] + P_discharge_S[k1,ω1,t1] - P_charge_S[k1,ω1,t1] - EL[j1,ω1,t1] == -2.5  …  E1[ω1,t3] : -PD[t3] + EG_i[i1,ω1,t3] - PW_curt[q1,ω1,t3] + P_discharge_S[k1,ω1,t3] - P_charge_S[k1,ω1,t3] - EL[j1,ω1,t3] == -6
 E1[ω2,t1] : -PD[t1] + EG_i[i1,ω2,t1] - PW_curt[q1,ω2,t1] + P_discharge_S[k1,ω2,t1] - P_charge_S[k1,ω2,t1] - EL[j1,ω2,t1] == -6       E1[ω2,t3] : -PD[t3] + EG_i[i1,ω2,t3] - PW_curt[q1,ω2,t3] + P_discharge_S[k1,ω2,t3] - P_charge_S[k1,ω2,t3] - EL[j1,ω2,t3] == -3.5
 E1[ω3,t1] : -PD[t1] + E

In [23]:
@constraints(model, begin
    # Energy generation constraint of power plants --> min = 1, max = 5
    E2[i in I, ω in Ω, t in T], v[i,ω,t] * 1 <= EG_i[i,ω,t] 
    E3[i in I, ω in Ω, t in T], EG_i[i,ω,t] <= v[i,ω,t] * 5  

    # Energy demand constraints of flexible loads --> min = 0.5, max = 2 (EL in book), at least 2.5 over all 3 periods  
    E4[j in J, ω in Ω, t in T], 0.5 <= EL[j,ω,t]  
    E5[j in J, ω in Ω, t in T], EL[j,ω,t] <= 2
    E6[j in J, ω in Ω], sum(EL[j,ω,t] for t in T) >= 2.5

    # Energy level of the battery

    # Consider battery SOC when starting
    E7[k in K, ω in Ω], ES[k,ω,"t1"] == 0.4 + P_charge_S[k,ω,"t1"] - P_discharge_S[k,ω,"t1"]
    E8[k in K, ω in Ω, t in T[2:end]], ES[k, ω, t] == ES[k, ω, previous_time_period(t, T)] + P_charge_S[k, ω, t] - P_discharge_S[k, ω, t]
    # Max and min SOC
    E9[k in K, ω in Ω, t in T], 0.2 <= ES[k, ω, t]
    E10[k in K, ω in Ω, t in T], ES[k, ω, t] <= 1 

    # Max and min (dis)charging rates

    E11[k in K, ω in Ω, t in T], 0 <= P_charge_S[k, ω, t]
    E12[k in K, ω in Ω, t in T], P_charge_S[k, ω, t] <= 0.3  
    E13[k in K, ω in Ω, t in T], 0 <= P_discharge_S[k, ω, t]
    E14[k in K, ω in Ω, t in T], P_discharge_S[k, ω, t] <= 0.5
        
    # Power generated by wind power unit in different scenarios and time steps
    cost_function[i in I, ω in Ω, t in T], C[i, ω, t] == bG * EG_i[i, ω, t] + cG
    utility_function[j in J, ω in Ω, t in T], U[j, ω, t] == bL * (EL[j, ω, t]) + cL
        
    # Maximum wind curtailment of wind power unit
    E24[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] >= 0
    E25[q in Q, ω in Ω, t in T], PW_curt[q,ω,t] <= PW[q,ω,t] 

end)

(3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, ["i1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, "t1"] =
 E2[i1,ω1,t1] : -EG_i[i1,ω1,t1] + v[i1,ω1,t1] <= 0  …  E2[i1,ω3,t1] : -EG_i[i1,ω3,t1] + v[i1,ω3,t1] <= 0

[:, :, "t2"] =
 E2[i1,ω1,t2] : -EG_i[i1,ω1,t2] + v[i1,ω1,t2] <= 0  …  E2[i1,ω3,t2] : -EG_i[i1,ω3,t2] + v[i1,ω3,t2] <= 0

[:, :, "t3"] =
 E2[i1,ω1,t3] : -EG_i[i1,ω1,t3] + v[i1,ω1,t3] <= 0  …  E2[i1,ω3,t3] : -EG_i[i1,ω3,t3] + v[i1,ω3,t3] <= 0, 3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessTh

### Optimization Objective

In [24]:
@objective(model, Max,
    sum(
    ( λ[t] * PD[t]) + sum(π[ω] * ( sum( U[j,ω,t] for j in J) - ( sum( C[i,ω,t] for i in I))) for ω in Ω) for t in T))


20 PD[t1] + 0.2 U[j1,ω1,t1] - 0.2 C[i1,ω1,t1] + 0.3 U[j1,ω2,t1] - 0.3 C[i1,ω2,t1] + 0.5 U[j1,ω3,t1] - 0.5 C[i1,ω3,t1] + 80 PD[t2] + 0.2 U[j1,ω1,t2] - 0.2 C[i1,ω1,t2] + 0.3 U[j1,ω2,t2] - 0.3 C[i1,ω2,t2] + 0.5 U[j1,ω3,t2] - 0.5 C[i1,ω3,t2] + 45 PD[t3] + 0.2 U[j1,ω1,t3] - 0.2 C[i1,ω1,t3] + 0.3 U[j1,ω2,t3] - 0.3 C[i1,ω2,t3] + 0.5 U[j1,ω3,t3] - 0.5 C[i1,ω3,t3]

In [25]:
print(model)

In [26]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
39 rows, 66 cols, 132 nonzeros
30 rows, 57 cols, 150 nonzeros
27 rows, 45 cols, 117 nonzeros
27 rows, 45 cols, 117 nonzeros

Solving MIP model with:
   27 rows
   45 cols (9 binary, 0 integer, 0 implied int., 36 continuous)
   117 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   927             -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   819.5           672.5             21.86%        0      0      0        18     0.0s

22.2% inactive integer columns, restarting
Model after restart has 25 rows, 42 cols (7 bin., 0 int., 0 impl., 35 cont.), and 104 nonzeros

         0       0         0   0.00%   819.

In [27]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [28]:
objective_value(model)

819.5

In [29]:
value.(PW_curt)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["q1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 0.0  0.0  0.0

[:, :, "t2"] =
 0.0  0.0  0.0

[:, :, "t3"] =
 0.0  0.0  0.0

In [30]:
value.(PD)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, ["t1", "t2", "t3"]
And data, a 3-element Vector{Float64}:
 4.7
 6.1
 6.0

In [31]:
value.(ES)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["k1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 0.7  0.7  0.7

[:, :, "t2"] =
 0.2  0.2  0.2

[:, :, "t3"] =
 0.2  0.2  0.2

In [32]:
value.(EL)

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, ["j1"]
    Dimension 2, ["ω1", "ω2", "ω3"]
    Dimension 3, ["t1", "t2", "t3"]
And data, a 1×3×3 Array{Float64, 3}:
[:, :, "t1"] =
 2.0  2.0  2.0

[:, :, "t2"] =
 2.0  2.0  0.5

[:, :, "t3"] =
 2.0  2.0  0.5